In [1]:
# ───────────────────────────── IMPORTS ─────────────────────────────
import pandas as pd
from datetime import datetime, timedelta
from extract.queries import fetch_ka_window, fetch_cases_window, last_n_months_window
from transform.clean import clean_articles, clean_cases
from transform.metrics import (
    close_reason_ratio,
    get_all_regions_valid_cases_and_ratios,
    median_days_to_publish,
    count_created_articles,
    count_published_articles,
    open_cases_week_over_week_by_region,
    osp_kcs_engagement,
)
from src.visualize.charts import format_kcs_engagement_by_region
from visualize.charts import (
    close_reason_overall_pie,
    close_reason_distribution_pie,
    plot_valid_cases_ratio_stacked,
    plot_articles_created_per_employee_by_region,
    plot_articles_created_per_coach_by_region,
    plot_open_cases_week_over_week_by_region,
)



C:\dev\Projects\kcs_report\.venv\Lib\site-packages\snowflake\connector\options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (20.0.0), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
# ───────────────────────────── DATE CALCULATIONS ─────────────────────────────
now = pd.Timestamp.now()
first_of_this_month = now.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
last_month_end = first_of_this_month - pd.Timedelta(seconds=1)
last_month_start = last_month_end.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
start = last_month_start
end = last_month_end

# Convert to string for DB compatibility
start_str = start.strftime("%Y-%m-%d %H:%M:%S")
end_str = end.strftime("%Y-%m-%d %H:%M:%S")



In [3]:
# ───────────────────────────── DATA FETCH & CLEAN ─────────────────────────────
df_ka = fetch_ka_window(start_str, end_str)   # DataFrame born here
df_cases = fetch_cases_window(start_str, end_str) # DataFrame born here

df_ka_clean = clean_articles(df_ka)
df_cases_clean = clean_cases(df_cases)



In [5]:
# ───────────────────────────── CHARTS & METRICS ─────────────────────────────
# # Show the overall close reason ratio as a pie chart
fig = close_reason_overall_pie(df_cases_clean)
fig.show()

# fig = close_reason_distribution_pie(df_cases_clean)
# fig.show()

# plot_valid_cases_ratio_stacked(df_cases_clean)

# created_count = count_created_articles(df_ka_clean, start, end)
# published_count = count_published_articles(df_ka_clean, start, end)
# print(f"Unique created articles in last month: {created_count}")
# print(f"Unique published articles in last month: {published_count}")

# median_days_to_publish = median_days_to_publish(df_ka_clean)
# print("Median days to publish:", median_days_to_publish)

# fig = plot_articles_created_per_employee_by_region(df_ka_clean, start, end)
# fig.show()

# fig = plot_articles_created_per_coach_by_region(df_ka_clean, start, end)
# fig.show()

plot_open_cases_week_over_week_by_region(df_cases_clean)

result = osp_kcs_engagement(df_cases_clean)
# print("Filtered count:", result["count"])
# print("Close reason ratio:", result["close_reason_ratio"])



AttributeError: 'tuple' object has no attribute 'show'

In [4]:
# ───────────────────────────── PDF EXPORT ─────────────────────────────
from src.report.exporter import export_report_pdf

# --- Generate all required figures ---
fig1, close_reason_overall_pie_insight = close_reason_overall_pie(df_cases_clean)
fig2 = close_reason_distribution_pie(df_cases_clean)
fig3, plot_valid_cases_ratio_stacked_insight = plot_valid_cases_ratio_stacked(df_cases_clean)
fig4 = plot_articles_created_per_employee_by_region(df_ka_clean, start, end)
fig5 = plot_articles_created_per_coach_by_region(df_ka_clean, start, end)
fig6 = plot_open_cases_week_over_week_by_region(df_cases_clean)

# --- Calculate metrics and insights ---
created_count = count_created_articles(df_ka_clean, start, end)
published_count = count_published_articles(df_ka_clean, start, end)
median_days = median_days_to_publish(df_ka_clean)
osp_result = osp_kcs_engagement(df_cases_clean)

figure_tuples = [
    ("Close Reason Overall Pie", fig1, close_reason_overall_pie_insight),
    ("Close Reason Distribution", fig2, "Knowledge Action: The breakdown of closed cases by the type of Knowledge Action taken"),
    (
        "Valid Cases Ratio Stacked",
        fig3,
        (
            "Top 3 Regions by Close Reason Ratio:\n" +
            "\n".join([
                f"{ordinal} place: {region}: {ratio:.1%}"
                for ordinal, (region, ratio) in zip(
                    ["1st", "2nd", "3rd"],
                    sorted(plot_valid_cases_ratio_stacked_insight.items(), key=lambda item: item[1], reverse=True)[:3]
                )
            ])
            if isinstance(plot_valid_cases_ratio_stacked_insight, dict) and plot_valid_cases_ratio_stacked_insight
            else "No region data available."
        )
    ),
    ("Articles Created per Employee by Region", fig4, f"Unique created articles in last month: {created_count}"),
    ("Articles Created per Coach by Region", fig5, f"Unique published articles in last month: {published_count}"),
    ("Open Cases Week over Week by Region", fig6, f"")
]

export_report_pdf(figure_tuples, output_path="kcs_report.pdf")

PDF report saved to kcs_report.pdf
